In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Dropping Indices column because it is not needed, and date column because there is a column for day, month, and year separately.
weather = pd.read_csv("/kaggle/input/saudi-arabia-weather-history/weather-sa-2017-2019-clean.csv")
weather = weather.drop(columns=['date','Unnamed: 0'])
listweather = list()
listcities = list()
# Numbering the cities with the occurence
for i,x in enumerate(np.unique(weather[['city']].values)):
    listcities.append(x)
    weather = weather.replace(x,i)
# Numbering the weather types with their occurence
for i,x in enumerate(np.unique(weather[['weather']].values)):
  listweather.append(x)
  weather = weather.replace(x,i)
# Casting the hour into int
for i,x in enumerate(np.unique(weather[['time']].values)):
  index = x.find(':')
  weather = weather.replace(x,int(x[:index]))

weather['humidity'] = weather['humidity'].str.replace("%","")
# Casting the humidity to float
weather['humidity'] = weather['humidity'].astype('float')
weather['humidity'] = weather['humidity']/100.0
# Replacing the 17 missing values with 0s
weathertesting = weather.replace(np.nan,0)

In [ ]:
# Seperating the set
X = weathertesting.drop(columns='temp') 
y = weathertesting.temp

In [ ]:
X.shape

In [ ]:
# Importing the needed libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.metrics import explained_variance_score
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20)

In [ ]:
RF = RandomForestRegressor(n_estimators=300, random_state=0)
RF.fit(xtrain,ytrain)
y_pred = RF.predict(xtest)
print("Score: ",explained_variance_score(ytest, y_pred))

# Run the cell below to get the fillable form
## Note: You only need to run it once

In [ ]:
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider,widgets

city = widgets.Dropdown(
    options=listcities,
    disabled=False,
)
datepick = widgets.DatePicker(
    description='Pick a Date',
    disabled=False
)


hour = widgets.IntSlider(
    value=0,
    min=0,
    max=23,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
minute = widgets.IntSlider(
    value=0,
    min=0,
    max=59,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

weathertype = widgets.Combobox(
    placeholder='Choose weather type',
    options=listweather,
    ensure_option=True,
    disabled=False
)

wind = widgets.BoundedIntText(
    value=0,
    min=0,
    max=100,
    step=1,
    disabled=False
)

humidity = widgets.BoundedFloatText(
    value=0.5,
    min=0,
    max=1,
    step=0.01,
    disabled=False
)

barometer = widgets.BoundedIntText(
    value=1000,
    min=0,
    max=1500,
    step=1,
    disabled=False
)

visibility = widgets.BoundedIntText(
    value=0,
    min=0,
    max=1000,
    step=1,
    disabled=False
)


form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value='City:'),
         city], layout=form_item_layout),
     Box([Label(value='Date:'),
         datepick], layout=form_item_layout),
    Box([Label(value='Hour:'),
         hour], layout=form_item_layout),
    Box([Label(value='Minute:'),
         minute], layout=form_item_layout),
    Box([Label(value='Weather:'),
         weathertype], layout=form_item_layout),
    Box([Label(value='Wind:'),
         wind], layout=form_item_layout),
    Box([Label(value='Humidity:'),
         humidity], layout=form_item_layout),
    Box([Label(value='Barometer:'),
         barometer], layout=form_item_layout),
    Box([Label(value='Visibility:'),
    visibility], layout=form_item_layout),
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))
form

# After configuring the form above, run the next cell
## Note: You do not have to run the cell above after updating the values

In [ ]:
if datepick.value == None:
    print("Please pick a date")
elif weathertype.value not in listweather:
    print("Please pick a weather from the combobox")
else:
    year = str(datepick.value).split('-')[0]
    month = str(datepick.value).split('-')[1]
    day = str(datepick.value).split('-')[2]
    details = [listcities.index(city.value),hour.value,int(year),int(month),int(day),hour.value,minute.value,listweather.index(weathertype.value),wind.value,humidity.value,barometer.value,visibility.value]
    print(int(RF.predict([details])),"C")